# Applied Dash Plotly to visualize Covid-19 Dashboard

## Import Library

In [69]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

## Download data

In [4]:
url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url_recovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

confirmed = pd.read_csv(url_confirmed)
deaths = pd.read_csv(url_deaths)
recovered = pd.read_csv(url_recovered)

In [42]:
confirmed.sample(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
111,NaN,Egypt,26.820553,30.802498,0,0,0,0,0,0,...,515698,515698,515698,515698,515698,515698,515698,515698,515759,515759
287,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,343012,343012,343079,343079,343079,343135,343135,343135,343135,343135
121,French Polynesia,France,-17.679700,149.406800,0,0,0,0,0,0,...,77957,77957,77957,77957,77957,77957,77957,77957,78055,78055
48,Nova Scotia,Canada,44.682000,-63.744300,0,0,0,0,0,0,...,140492,140492,140492,140492,140492,140492,140793,140793,140793,140793
119,NaN,Finland,61.924110,25.748151,0,0,0,0,0,0,...,1462169,1462169,1462976,1462976,1462976,1462976,1462976,1462976,1462976,1463644
105,NaN,Denmark,56.263900,9.501800,0,0,0,0,0,0,...,3403360,3403472,3403594,3403707,3403707,3403707,3403958,3404128,3404268,3404407
144,NaN,Holy See,41.902900,12.453400,0,0,0,0,0,0,...,29,29,29,29,29,29,29,29,29,29
248,NaN,Syria,34.802075,38.996815,0,0,0,0,0,0,...,57467,57467,57467,57467,57467,57467,57467,57467,57467,57467
159,NaN,Kenya,-0.023600,37.906200,0,0,0,0,0,0,...,342912,342919,342919,342919,342919,342919,342919,342932,342937,342937
266,British Virgin Islands,United Kingdom,18.420700,-64.640000,0,0,0,0,0,0,...,7305,7305,7305,7305,7305,7305,7305,7305,7305,7305


## Preprocessing Data

### Melting data

In [37]:
confirmed_date = confirmed.columns[4:] 
confirmed_geo = confirmed.columns[:4]
total_confirmed = confirmed.melt(id_vars=confirmed_geo, value_vars= confirmed_date,value_name='Confirmed',var_name='Date')
# total_confirmed
deaths_date = deaths.columns[4:]
deaths_geo = deaths.columns[:4]
total_deaths = deaths.melt(id_vars=confirmed_geo, value_vars=deaths_date, value_name='Deaths',var_name='Date')

recovered_date = recovered.columns[4:]
recovered_geo = recovered.columns[:4]
total_recovered= recovered.melt(id_vars=recovered_geo, value_vars=recovered_date, value_name='Recovered',var_name='Date')

### Merging Data

In [38]:
covid_data = total_confirmed.merge(total_deaths, on=confirmed_geo.to_list() + ['Date'], how='left')
covid_data = covid_data.merge(total_recovered, on = recovered_geo.to_list() + ['Date'], how='left')

In [43]:
covid_data.sample(10)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
251538,NaN,Dominica,15.415000,-61.371000,2022-06-10,14638,67,0.0
209019,Hubei,China,30.975600,112.270700,2022-01-14,68320,4512,0.0
128851,NaN,Sweden,60.128161,18.643501,2021-04-11,857401,13621,0.0
7215,NaN,Uruguay,-32.522800,-55.765800,2020-02-15,0,0,0.0
152204,NaN,Montenegro,42.708678,19.374390,2021-07-01,100272,1613,98371.0
175739,NaN,Bhutan,27.514200,90.433600,2021-09-21,2599,3,0.0
108906,NaN,Sri Lanka,7.873054,80.771797,2021-02-01,64983,323,58075.0
171527,NaN,Iran,32.427908,53.688046,2021-09-06,5156986,111257,0.0
304998,Faroe Islands,Denmark,61.892600,-6.911800,2022-12-12,34658,28,0.0
235310,Guangdong,China,23.341700,113.424400,2022-04-15,6847,8,0.0


In [40]:
covid_data['Date'] = pd.to_datetime(covid_data['Date'])

/tmp/ipykernel_7747/3902955790.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  covid_data['Date'] = pd.to_datetime(covid_data['Date'])


In [41]:
covid_data.isna().sum()

Province/State    226314
Country/Region         0
Lat                 2286
Long                2286
Date                   0
Confirmed              0
Deaths                 0
Recovered          24003
dtype: int64

In [44]:
covid_data['Recovered'] = covid_data['Recovered'].fillna(0)

In [47]:
covid_data['Active'] = covid_data['Confirmed'] - covid_data['Deaths'] - covid_data['Recovered']

In [51]:
covid_data_1 = covid_data.groupby(['Date'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()
covid_data_1

,Date,Confirmed,Deaths,Recovered,Active
0,2020-01-22,557,17,30.0,510.0
1,2020-01-23,657,18,32.0,607.0
2,2020-01-24,944,26,39.0,879.0
3,2020-01-25,1437,42,42.0,1353.0
4,2020-01-26,2120,56,56.0,2008.0
...,...,...,...,...,...
1138,2023-03-05,676024901,6877749,0.0,669147152.0
1139,2023-03-06,676082941,6878115,0.0,669204826.0
1140,2023-03-07,676213378,6879038,0.0,669334340.0
1141,2023-03-08,676392824,6880483,0.0,669512341.0


In [53]:
covid_data_2 = covid_data.groupby(['Date', 'Country/Region'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()
covid_data_2

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
229738,2023-03-09,West Bank and Gaza,703228,5708,0.0,697520.0
229739,2023-03-09,Winter Olympics 2022,535,0,0.0,535.0
229740,2023-03-09,Yemen,11945,2159,0.0,9786.0
229741,2023-03-09,Zambia,343135,4057,0.0,339078.0


In [65]:
covid_data_dict = covid_data[['Country/Region', 'Lat', 'Long']]
list_locations = covid_data_dict.set_index('Country/Region')[['Lat', 'Long']].T.to_dict()

/tmp/ipykernel_7747/3770569221.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  list_locations = covid_data_dict.set_index('Country/Region')[['Lat', 'Long']].T.to_dict()


## Creating Dashboard

Continue at main.py

In [66]:
# app = dash.Dash(__name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}])

In [70]:
# app.layout = html.Div([
#     html.Div([
#         html.Div([
#             html.Img(src=app.get_asset_url('OIP.jpeg'),
#                      id='corona-image',
#                      style={
#                          'height':'60px',
#                          'width': 'auto',
#                          'margin-bottom':'25px',
#                      })
#         ])
#     ])
# ])

In [72]:
# if __name__ == '__main__':
#     app.run_server()

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

---------------------------------------------------------------------------
NotFound                   